# Programa para generar planes de mantenimiento, a partir del URL, o desde los archivos CSV descargados

Modificaciones realizadas 12/2025
* Generacion a partir de Lineas Alta Demanda
* Lineas Baja Demanda




## Cargar librerias necesarias

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from pandas import DataFrame, Series

## Cargar hoja Equipos-Planes

In [8]:
## Importar CSV de Estructura funcional Ubicacion
import pandas as pd
import re
import os

# This function will convert the url to a download link
def convert_gsheets_url(u):
    try:
        worksheet_id = u.split('#gid=')[1]
    except:
        # Couldn't get worksheet id. Ignore it
        worksheet_id = None
    u = re.findall('https://docs.google.com/spreadsheets/d/.*?/',u)[0]
    u += 'export'
    u += '?format=csv'
    if worksheet_id:
        u += '&gid={}'.format(worksheet_id)
    return u

URL = 'https://docs.google.com/spreadsheets/d/1yOaSeqRBr1FW6tvFMi_Y-s4011cKBoyiWU5dTMlujrU/edit?gid=1115106678#gid=1115106678'     # introducir desde consola
csv_raw = convert_gsheets_url(URL)
print(csv_raw)


# Si existe el CSV importarlo, sino cargar de la URL
filename = "csv/Equipos-PlanesLBD.csv"

if os.path.exists(filename):
    print("Leyendo desde Archivo")
    df_equipos = pd.read_csv(filename)    
else:
    print("Leyendo desde URL")
    df_equipos = pd.read_csv(csv_raw,header=3)
    df_equipos.to_csv(filename,index=False)
print(df_equipos.shape)


https://docs.google.com/spreadsheets/d/1yOaSeqRBr1FW6tvFMi_Y-s4011cKBoyiWU5dTMlujrU/export?format=csv&gid=1115106678
Leyendo desde Archivo
(2752, 111)


In [9]:
df_equipos.head()

,id,Sist,Subs,Equ,Comp,Equipo,Componente,Parte,Tipo_equipo,Criticidad,...,PL-S1-R1,PL-S1-TO,PL-S1-CT,PL-S1-LAA,PL-LIN-VH,PL-EDF-E16,PL-EDF-EFM,PL-EDF-EMR,PL-S1-I1-TMC,Unnamed: 110
0,1.0,ACC,NaN,NaN,NaN,ACCIONAMIENTO,NaN,NaN,Sistema,NaN,...,True,False,False,False,False,False,False,False,False,FALSE
1,1.0,ACC,PRI,NaN,NaN,ACCIONAMIENTO PRINCIPAL,NaN,NaN,Subsistema,NaN,...,False,False,False,False,False,False,False,False,False,FALSE
2,1.0,ACC,PRI,MEPA,NaN,MOTOR ELÉCTRICO PRINCIPAL ACOPLADO,NaN,NaN,Equipo,MUY ALTA,...,False,False,False,False,False,False,False,False,False,NaN
3,2.0,ACC,PRI,MEPA,MEL,NaN,MOTOR ELECTRICO,NaN,Componente,MUY ALTA,...,False,False,False,False,False,False,False,False,False,NaN
4,3.0,ACC,PRI,MEPA,RM,NaN,REFRIGERACION DE MOTOR,NaN,Componente,MUY ALTA,...,False,False,False,False,False,False,False,False,False,NaN


In [11]:
## Importar CSV de Planes de Mantto
URL = 'https://docs.google.com/spreadsheets/d/1OkECu7qNfGZxX_rc2RDbaz0A-oE_gUwJ0P2tjU_x-q0/edit#gid=1199302294'     # Introducir desde consola
csv_raw = convert_gsheets_url(URL)
print(csv_raw)
# Si existe el CSV importarlo, sino cargar de la URL
filename = "csv/MaestroActividadesLBD.csv"
if os.path.exists(filename):
    print("Leyendo desde Archivo")
    df_planes = pd.read_csv(filename)        
else:
    print("Leyendo desde URL")
    df_planes = pd.read_csv(csv_raw,header=3)
    df_planes.to_csv(filename,index=False)
print(df_planes.shape)
# Eliminar si tiene valores nulos
df_planes = df_planes [df_planes .Plan.notnull()]
print(df_planes.shape)

https://docs.google.com/spreadsheets/d/1OkECu7qNfGZxX_rc2RDbaz0A-oE_gUwJ0P2tjU_x-q0/export?format=csv&gid=1199302294
Leyendo desde Archivo
(1397, 38)
(1396, 38)


In [12]:
df_update = df_planes[~df_planes.Accion.isna()]     # Filtrar solo las columnas de dfPlanes que no tienen accion


In [13]:
df_update = df_planes[~df_planes.Accion.isna()]     # Filtrar solo las columnas de dfPlanes que no tienen accion

# eliminar las tareas basadas en condicion, #TODO: Analizar este cambio. }
df_update = df_update[df_update ['Especialidad']!='BASADA EN CONDICION']

# En el dataframe de planes, en la fila del plan se coloca si tiene alguna frecuencia al inicio del plan
gb = df_update.groupby('Plan',sort=False).any()
gb = gb.reset_index()
# Concatenamos los dos dataframes por filas
df_planes = pd.concat([gb,df_update], axis=0, ignore_index=True)

In [6]:
# En caso no este instalado, no se puede manipular excel
#!pip install openpyxl

In [7]:
# Hacemos un join de los dos DataFrames por la izquierda
df_merged = pd.merge(df_equipos, df_planes, how='left', left_on= 'Plan', right_on= 'Plan',suffixes=('_equipo', '_plan'))
# aplicar una funcion en una columna de un dataframe dependiendo si la columna que actualizamos no estan dentro de ['Actividad','Tarea'], si no copiar el valor de la columna 'tipo_equipo', si tiene valor, mantener el valor de la columna 'tipo_plan'
df_merged['Tipo_plan']=df_merged.apply(lambda row: row['Tipo_equipo'] if  row['Tipo_plan']  not in ['Actividad','Tarea']  else  row ['Tipo_plan'] , axis='columns')

# realizar una funcion apply en el dataframe df_merge que evalue el contenido de la columna 'Tipo_Plan', si su valor es 'Componente' que vacie el contenido de la columna 'Accion' y la columna 'Actividad', de lo contrario que mantenga su valor
df_merged['Accion'] = df_merged.apply(lambda row : '' if row['Tipo_plan'] == 'Componente' else row['Accion'],axis='columns')
df_merged['Actividad'] = df_merged.apply(lambda row : '' if row['Tipo_plan'] == 'Componente' else row['Actividad'],axis='columns')

#Exportar dataframe a Excel para visualizacion 
# df_merged.to_excel('xls/merged.xlsx') # 12Mb el archivo, dificil de analizar

In [8]:
# Leer dataframe desde archivo excel
#df = df_merged.copy(deep=True) # pd.read_excel('xls/test_group.xlsx', sheet_name='Sheet1', index_col='id')

# EQUIPOS
index = df_merged[df_merged['Tipo_plan']=='Equipo'].index.to_list()
#print(len(index))
# obtener las columnas que se desean actualizar
cols = ['Sist','Subs','Equ','Comp','D', 'S', 'M', '2M', 'T', '4M', 'SE', '8M', 'A', '1.5A', '2A', '3A', '4A', '5A', '6A', '9A', '10A', '1000', '6000', '22500', '40000', '55000']
m_cols = df_merged.columns.to_list()
update_cols = [i for i in m_cols if i not in cols]
copy_cols = update_cols + ['Sist','Subs','Equ','Comp']
print (copy_cols)


df_g = df_merged[df_merged['Tipo_plan'].isin(['Equipo','Componente'])][cols]

# Agrupar por Equ
gb= df_g.groupby(['Equ'],sort=False).any().reset_index()
gb['id'] = index
gb.set_index('id',inplace=True)
#print (gb)

#TODO: Necesario mejorar el rendimiento del algoritmo aparecen mensajes de warning
#gb = pd.concat([gb,df[cols]],axis = 1)
gb[copy_cols] = df_merged.loc[gb.index,copy_cols]
df_merged.loc[gb.index,:] = gb

# Exportar excel para visualizacion
#df.head(302).to_excel('xls/df.xlsx')

# Subsistema
index = df_merged[df_merged['Tipo_plan']=='Subsistema'].index.to_list()
print((index))
df_g = df_merged[df_merged['Tipo_plan'].isin(['Equipo','Subsistema'])][cols]

# Agrupar por Subs
gb= df_g.groupby(['Subs'],sort=False).any().reset_index()
gb['id'] = index
gb.set_index('id',inplace=True)
# dar formato para copiar
gb[copy_cols] = df_merged.loc[gb.index,copy_cols]
df_merged.loc[gb.index,:] = gb

#df.head(302).to_excel('xls/df.xlsx')

############# Sistema
index = df_merged[df_merged['Tipo_plan']=='Sistema'].index.to_list()
#print(index)
df_g = df_merged[df_merged['Tipo_plan'].isin(['Sistema','Subsistema'])][cols]
# Agrupar por Sist
gb= df_g.groupby(['Sist'],sort=False).any().reset_index()
gb['id'] = index
gb.set_index('id',inplace=True)
# dar formato para copiar
gb[copy_cols] = df_merged.loc[gb.index,copy_cols]
df_merged.loc[gb.index,:] = gb

# TODO: Reemplazar los valores en la columna 'Parada' por 'null' cuando 'Tipo_plan' sea igual a 'Componente'
df_merged.loc[df_merged['Tipo_plan'] == 'Componente', ['Parada','Relevancia','Especialidad']] = None

# TODO: Es necesario que los valores asignados como tareas se quite la relevancia para que no sean tomados en cuenta en la ponderación del check
# Filtrar las filas donde Tipo_plan es igual a 'Tarea' y establecer la columna 'relevancia' a None
df_merged.loc[df_merged['Tipo_plan'] == 'Tarea', ['Relevancia','Parada','Especialidad']] = None

df_merged.head(100).to_excel('xls/df_merged.xlsx')
#print(df)




['Equipo', 'Componente', 'Parte', 'Tipo_equipo', 'Criticidad', 'Sigla', 'Tag', 'Plan', 'AMEF', 'RO', 'RO-S2-R1', 'RO-S2-M1', 'RO-S2-TO', 'RO-S2-CT', 'RO-S2-LAA', 'RO-S1-M1', 'RO-S1-R1', 'RO-S1-TO', 'RO-S1-CT', 'RO-S1-LAA', 'RO-LIN-VH', 'RO-EDF-ECT', 'RO-EDF-ECM', 'RO-EDF-EJL', 'RO-S2-R1-TMC', 'AM', 'AM-S1-R1', 'AM-S1-M1', 'AM-S1-R2', 'AM-S1-TO', 'AM-S1-CT', 'AM-S1-CT.1', 'AM-S1-LAA', 'AM-S2-R1', 'AM-S2-M1', 'AM-S2-TO', 'AM-S2-CT', 'AM-S2-LAA', 'AM-LIN-VH', 'AM-EDF-ELB', 'AM-EDF-ESS', 'AM-EDF-EBA', 'AM-EDF-EMR', 'AM-S1-M1-TMC', 'VE', 'VE-S2-R1', 'VE-S2-I1', 'VE-S2-M1', 'VE-S2-TO', 'VE-S2-CT', 'VE-S2-LAA', 'VE-S1-M1', 'VE-S1-R1', 'VE-S1-TO', 'VE-S1-CT', 'VE-S1-LAA', 'VE-LIN-VH', 'VE-EDF-EIR', 'VE-EDF-EOB', 'VE-EDF-EAO', 'VE-EDF-ELB', 'VE-S2-I1-TMC', 'AZ', 'AZ-S1-M1', 'AZ-S1-I1', 'AZ-S1-I2', 'AZ-S1-R1', 'AZ-S1-TO', 'AZ-S1-CT', 'AZ-S1-LAA', 'AZ-S2-R1', 'AZ-S2-M1', 'AZ-S2-TO', 'AZ-S2-CT', 'AZ-S2-LAA', 'AZ-LIN-VH', 'AZ-EDF-EDJ', 'AZ-EDF-ELD', 'AZ-EDF-ELP', 'AZ-EDF-EUP', 'AZ-EDF-ERS', 'AZ-S1-

C:\Users\mantenimiento\AppData\Local\Temp\ipykernel_16288\1423018278.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  gb[copy_cols] = df_merged.loc[gb.index,copy_cols]
C:\Users\mantenimiento\AppData\Local\Temp\ipykernel_16288\1423018278.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  gb[copy_cols] = df_merged.loc[gb.index,copy_cols]
C:\Users\mantenimiento\AppData\Local\Temp\ipykernel_16288\1423018278.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many tim

In [9]:
# Abrir archivo json

import json
with open('./csv/Lineas.json') as f:
    diccionario_lineas = json.load(f)

frecuencias=['D', 'S', 'M', '2M', 'T', '4M', 'SE', '8M', 'A', '1.5A', '2A', '3A', '4A', '5A', '6A', '9A', '10A', '1000', '6000', '22500', '40000', '55000']

# df_merged[ ['id', 'Cod', 'Accion', 'Actividad', 'Tipo_plan', 'Parada', 'Relevancia', 'Especialidad']+frecuencias]
head = ['id', 'Cod', 'Equipo','Componente', 'Accion', 'Actividad', 'Tipo_plan', 'Parada', 'Relevancia', 'Especialidad'] 

version = "_v2_1_2023"  # importar desde consola

for linea in diccionario_lineas['Lineas']:
    print (linea['Linea'])
    print ( head + frecuencias + linea['Columnas'])       
    # exportar plan maestro de linea
    df_linea = df_merged[head + frecuencias + linea['Columnas']]

    # A nivel de componente, solamente mantiene el primer valor, si encuentra mas adelante coloca ''
    # df_linea.loc[df_linea.duplicated(subset=['Componente']), 'Componente'] = '' 

    #Filtrar solamente los datos de la columna de la linea correspondiente, no todas las demas columnas
    df_linea = df_linea[df_linea[linea['Linea']] == True]
    ## Exportar Excel de Estructura - Planes    
    df_linea.to_excel(f"xls/{linea['Linea']}{version}.xlsx" , index= False)     
    print(df_linea.shape)        
    # break #TODO: En caso solo se genera de 1 linea

RO
['id', 'Cod', 'Equipo', 'Componente', 'Accion', 'Actividad', 'Tipo_plan', 'Parada', 'Relevancia', 'Especialidad', 'D', 'S', 'M', '2M', 'T', '4M', 'SE', '8M', 'A', '1.5A', '2A', '3A', '4A', '5A', '6A', '9A', '10A', '1000', '6000', '22500', '40000', '55000', 'RO', 'RO-S2-R1', 'RO-S2-M1', 'RO-S1-M1', 'RO-S1-R1']
(1237, 37)
AM
['id', 'Cod', 'Equipo', 'Componente', 'Accion', 'Actividad', 'Tipo_plan', 'Parada', 'Relevancia', 'Especialidad', 'D', 'S', 'M', '2M', 'T', '4M', 'SE', '8M', 'A', '1.5A', '2A', '3A', '4A', '5A', '6A', '9A', '10A', '1000', '6000', '22500', '40000', '55000', 'AM', 'AM-S1-R1', 'AM-S1-M1', 'AM-S1-R2', 'AM-S2-R1', 'AM-S2-M1']
(1639, 38)
VE
['id', 'Cod', 'Equipo', 'Componente', 'Accion', 'Actividad', 'Tipo_plan', 'Parada', 'Relevancia', 'Especialidad', 'D', 'S', 'M', '2M', 'T', '4M', 'SE', '8M', 'A', '1.5A', '2A', '3A', '4A', '5A', '6A', '9A', '10A', '1000', '6000', '22500', '40000', '55000', 'VE', 'VE-S2-R1', 'VE-S2-I1', 'VE-S2-M1', 'VE-S1-M1', 'VE-S1-R1']
(1628, 38)
A